In [1]:
import io
import pickle
from copy import deepcopy
from matplotlib import pyplot as plt
from PIL import Image
import subprocess
from subprocess import Popen, PIPE
import random
import imageio
import numpy as np

In [2]:
from matplotlib.backends.backend_template import FigureCanvas
def plot_to_PIL(fig, dpi=100):
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=dpi)
    buf.seek(0)
    pil_img = deepcopy(Image.open(buf))
    buf.close()
    return pil_img

def interpolate_images(img1, img2, factor):
    # Veic pāreju starp diviem attēliem
    # Factor 0 nozīmē ka pilnībā redzams tikai img1
    # Factor 1 nozīmē ka pilnībā redzams tikai img2
    # Tiek pieņemts ka attēliem ir vienādas dimensijas - ja tā nav - pats esi vainīgs
    # Tiek arī pieņemts ka abu attēlu feature vērtības ir apgabalā [0;+1] (kas ir arī SOM svaru īpašība)
    new_image = [
        [
            [img1[i][j][0]*(1-factor) + img2[i][j][0]*factor,
             img1[i][j][1]*(1-factor) + img2[i][j][1]*factor,
             img1[i][j][2]*(1-factor) + img2[i][j][2]*factor] for j in range(len(img1[0]))
        ] for i in range(len(img1))
    ]

    return new_image

def random_image(dimension):
    new_image = [[[random.random(),random.random(),random.random()] for j in range(dimension)] for i in range(dimension)]

    return new_image

# image ir NxN masīvs, kurā katrs elements ir vektors [x1, x2, x3]
def plot_som(image, meta_this_epoch=None, meta_max_epoch=None, meta_name=None):
    fig = plt.Figure(figsize=[6, 6])
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    fig.patch.set_facecolor('white')
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)

    fig.subplots_adjust(left = 0, right = 1, bottom = 0.1, top = 0.9)
    if meta_this_epoch is not None and meta_max_epoch is not None:
        ax.set_title("\"{}\" {:.1f}%/{} epohām".format(meta_name, 100.0*((meta_this_epoch+1)/meta_max_epoch), 2000))

    ax.imshow(image, interpolation='gaussian')
    #ax.axis('tight')
    #ax.axis('off')
    return fig

# image_list ir masīvs, kurš sastāv no iepriekš aprakstītā veida "images"
def som_animation(image_list, frames_per_second=30, seconds_per_epoch=1, name='correcthorsebatterystaple'):
    previous_image = None

    #fps, duration = frames_per_epoch, frames_per_epoch * (len(image_list)-1)
    #p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'mpeg4', '-qscale', '5', '-r', str(fps), 'video.mp4'], stdin=PIPE, stdout=PIPE, shell=True)
    writer = imageio.get_writer('{}.mp4'.format(name), fps=frames_per_second)
    for idx, img in enumerate(image_list):
        print("Rendering {}/{}".format(idx, len(image_list)))
        if previous_image is None:
            # Vajag uzstādīt pirmo attēlu, no kura notiks pāreja
            previous_image = img
            continue

        for i in range(round(frames_per_second * seconds_per_epoch)):
            interp_img = interpolate_images(previous_image, img, i/(frames_per_second * seconds_per_epoch))
            fig = plot_som(interp_img, meta_this_epoch=idx, meta_max_epoch=len(image_list), meta_name=name)
            pil_fig = plot_to_PIL(fig)
            #pil_fig.save(p.stdin, 'PNG')
            fig.clf()
            #img = createFrame(i)
            writer.append_data(np.array(pil_fig))
        previous_image = img
    writer.close()
    #p.stdin.close()
    #p.wait()


In [3]:
# Izveidot video visiem eksperimentiem

data = ["rate_0.1.pickle", "rate_0.3.pickle", "rate_0.5.pickle", "rate_0.7.pickle", "rate_0.9.pickle",
        "size_10.pickle", "size_30.pickle", "size_50.pickle"]

for d in data:
    list_x = pickle.load(open(d, "rb"))
    name = d.removesuffix(".pickle")
    plot_som(list_x[-1])
    som_animation(list_x, 30, 0.1, name=name)

Rendering 0/401
Rendering 1/401
Rendering 2/401
Rendering 3/401
Rendering 4/401
Rendering 5/401
Rendering 6/401
Rendering 7/401
Rendering 8/401
Rendering 9/401
Rendering 10/401
Rendering 11/401
Rendering 12/401
Rendering 13/401
Rendering 14/401
Rendering 15/401
Rendering 16/401
Rendering 17/401
Rendering 18/401
Rendering 19/401
Rendering 20/401
Rendering 21/401
Rendering 22/401
Rendering 23/401
Rendering 24/401
Rendering 25/401
Rendering 26/401
Rendering 27/401
Rendering 28/401
Rendering 29/401
Rendering 30/401
Rendering 31/401
Rendering 32/401
Rendering 33/401
Rendering 34/401
Rendering 35/401
Rendering 36/401
Rendering 37/401
Rendering 38/401
Rendering 39/401
Rendering 40/401
Rendering 41/401
Rendering 42/401
Rendering 43/401
Rendering 44/401
Rendering 45/401
Rendering 46/401
Rendering 47/401
Rendering 48/401
Rendering 49/401
Rendering 50/401
Rendering 51/401
Rendering 52/401
Rendering 53/401
Rendering 54/401
Rendering 55/401
Rendering 56/401
Rendering 57/401
Rendering 58/401
Renderi

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 600) to (608, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
IMAGEIO FFMPEG_WRITER WARNING: input ima

In [4]:
# Load epochs from SOM training
# list_x = pickle.load( open( "all_epochs.pickle", "rb" ) )

In [5]:
#list_x = [random_image(100) for i in range(2)]
# plot_som(list_x[-1])

In [6]:
#som_animation(list_x, 30, 0.05)